In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import re
import urllib.parse
import os
import time # Usamos 'time' para medir la velocidad de las consultas

url_base = "https://books.toscrape.com/" #pagina base


In [19]:
respuesta = requests.get(url_base)
#emite una solicitud HTTPGET a la URL indicada. Recupera los datos HTML que el servidor envía y los almacena en un objeto Python llamado respuesta

sopa = BeautifulSoup(respuesta.text , "html.parser") #analizador, le dice al beatifulsoup tiene que organizar lo obtenido 
#Permite interactuar con HTML de forma similar a como se interactúa con una página web mediante herramientas de desarrollo. 
#La biblioteca ofrece métodos intuitivos que permiten explorar el HTML recibido.
#Salida: El Árbol DOM (Document Object Model) o estructura jerárquica de nodos

categorias = {} #un diccionario donde guardaremos las categorias con sus links
lista = sopa.find("ul", class_="nav-list").find("ul").find_all("a")
# buscamos la etiqueta en donde se encuentran las categorias

for a in lista:
    nombre = a.get_text(strip=True)
    link = url_base + a["href"] #Referencia de Hipertexto, especifica la dirección o URL
    categorias[nombre] = link
# como en la lista ya tenemos todos los nombres de las categorias, guardamos su nombre y agregamos su link a la url base

# Guardar tabla de categorias
filas_de_categorias = [{"nombre": n} for n in categorias.keys()]  # no guardamos la URL en la tabla de categorías, solo el nombre

print("Categorías encontradas:", len(categorias))

Categorías encontradas: 50


In [21]:

def parse_price(price_text):
    if not price_text:
        return None
    s = price_text.strip()
    s = s.replace("\xa0", "").replace(" ", "")             # quitar espacios invisibles
    s = re.sub(r"[^\d\.,]", "", s)                         # dejar solo dígitos, punto y coma, substitución de expresión regular
    if "," in s and "." in s:
        s = s.replace(",", "")                             # coma como separador de miles
    elif "," in s and "." not in s:
        s = s.replace(",", ".")                            # coma como decimal
    try:
        return float(s)
    except Exception:
        return None
#una funcion para limpiar el precio de los libros


# diccionario para mapear rating texto -> int
calificaciones_numeros = {"One":1, "Two":2, "Three":3, "Four":4, "Five":5}

#recorrer cada categoria y extraer cada libro
fila_de_libros = [] # inicializamos un vector en donde se van a almacenar los datos
autores_unicos_scrapeados = set()
for nombre_categoria, url_categoria in categorias.items():
    print(f"\nExtrayendo libros de categoria : {nombre_categoria}")
    
    pagina_url = url_categoria  # empezamos por la primera página de la categoría

    while pagina_url:
        respuesta_categoria = requests.get(pagina_url) #pedimos una peticion para que podamos obtener el codigo de la pagina de cierta categoria
        sopa_categoria = BeautifulSoup(respuesta_categoria.text, "html.parser") # lo organizamos para poder manipularlos luego

        libros = sopa_categoria.find_all("article", class_ = "product_pod")
        #buscamos todos contenedores "article" en donde los elementos son las "clases product pod"

        for libro in libros:
            # título
            titulo = libro.h3.a["title"] #busco el contenedor h3 y luego bajo al contenedor a que ahi en donde esta el titulo
            
            # precio: limpiar y convertir a float
            precio_text = libro.find("p", class_="price_color").text.strip() # busco el contenedor p con la etiqueta de clase correspondiente y busca los textos de la misma
            precio_num = parse_price(precio_text)
            
            #Disponibilidad
            disponibilidad = libro.find("p", class_="instock availability").text.strip()
            
            # calificación: mapear la clase a int
            calificacion = libro.find("p", class_="star-rating")
            clasificacion_numero = None
            if calificacion:
                clases = calificacion.get("class", [])
                clasificacion_numero = next((calificaciones_numeros[c] for c in clases if c in calificaciones_numeros), None) #toma el generador como su primer argumento
                
            # buscar autor o autores en OpenLibrary por título (API pública)
            autores_lista = ["Desconocido"] 

            try:
                tit = urllib.parse.quote_plus(titulo) #codifica el string para usarlo en una URL
                autor_url = f"https://openlibrary.org/search.json?title={tit}"
                olr = requests.get(autor_url)
                
                if olr.ok: #sirve para saber si se pudo llegar a la pagina y no lanzo ningun error 
                    data = olr.json() #convierte ese texto en estructuras Python (dict/list) que son mucho más sencas de manejar que parsear HTML con BeautifulSoup
                    docs = data.get("docs") or [] #intentamos obtener docs, si no hay nada decuelve una lista vacia
                    
                    if docs:
                        #Obtenemos directamente la lista de nombres de autores
                        autores_api = docs[0].get("author_name") 
                        
                        if autores_api:
                            #Si la lista de la API NO está vacía, la usamos.
                            autores_lista = autores_api

            except Exception:
                # Si la API falla por completo, usamos la lista por defecto.
                autores_lista = ["Desconocido"] 
    
            autores_unicos_scrapeados.update(autores_lista)
            # guardar cada libro en la lista acumulada
            fila_de_libros.append({
                "titulo": titulo,
                "precio": precio_num,
                "disponibilidad": disponibilidad,
                "calificacion": clasificacion_numero,
                "autores_lista": autores_lista,
                "categoria": nombre_categoria  # guardamos el nombre de la categoría
            })
            
        # paginación: si existe "li.next" seguir a la siguiente página
        siguiente_linea = sopa_categoria.find("li", class_="next") #usca una etiqueta de lista (<li>) que tenga la clase CSS "next"
        if siguiente_linea and siguiente_linea.a: #Primero, verifica si se encontró el elemento, Luego, verifica si dentro de ese elemento <li> existe una etiqueta de anclaje (<a>), que es donde estará el enlace real a la siguiente página.
            siguiente_href = siguiente_linea.a["href"] #extrae el valor del atributo href 
            pagina_url = urllib.parse.urljoin(pagina_url, siguiente_href) #Toma la URL base actual (pagina_url) y la combina correctamente con la URL relativa (siguiente_href) para crear la URL completa y absoluta de la siguiente página.
        else:
            pagina_url = None # si no puede la pagina es none

print("Extracción completada. Libros totales:", len(fila_de_libros))
print("Autores únicos recolectados:", len(autores_unicos_scrapeados))



Extrayendo libros de categoria : Travel

Extrayendo libros de categoria : Mystery

Extrayendo libros de categoria : Historical Fiction

Extrayendo libros de categoria : Sequential Art

Extrayendo libros de categoria : Classics

Extrayendo libros de categoria : Philosophy

Extrayendo libros de categoria : Romance

Extrayendo libros de categoria : Womens Fiction

Extrayendo libros de categoria : Fiction

Extrayendo libros de categoria : Childrens

Extrayendo libros de categoria : Religion

Extrayendo libros de categoria : Nonfiction

Extrayendo libros de categoria : Music

Extrayendo libros de categoria : Default

Extrayendo libros de categoria : Science Fiction

Extrayendo libros de categoria : Sports and Games

Extrayendo libros de categoria : Add a comment

Extrayendo libros de categoria : Fantasy

Extrayendo libros de categoria : New Adult

Extrayendo libros de categoria : Young Adult

Extrayendo libros de categoria : Science

Extrayendo libros de categoria : Poetry

Extrayendo libr

In [ ]:
#creacion de las tablas
conexion = sqlite3.connect("libreria.db") #iniciamos la la bd
cursor = conexion.cursor() # creamos un cursor para ejecutar los comandos SQL

#La tabla de las categorias
cursor.execute("""
CREATE TABLE IF NOT EXISTS categorias(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT UNIQUE    
);
""")

#la tabla de los autores
cursor.execute("""
CREATE TABLE IF NOT EXISTS autores (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT UNIQUE
);
""")

# la tabla de los libros
cursor.execute("""
CREATE TABLE IF NOT EXISTS libros (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT,
    precio REAL,
    disponibilidad TEXT,
    calificacion INTEGER,
    categoria_id INTEGER,
    FOREIGN KEY (categoria_id) REFERENCES categorias(id)
);
""")

#la tabla de los libros con cada autor
cursor.execute("""
CREATE TABLE IF NOT EXISTS libro_autor (
    libro_id INTEGER,
    autor_id INTEGER,
    FOREIGN KEY (libro_id) REFERENCES libros(id),
    FOREIGN KEY (autor_id) REFERENCES autores(id),
    PRIMARY KEY (libro_id, autor_id)
);
""")
#El valor de libro_id debe  existir en la tabla libros, en la columna id
#Cada combinación de libro y autor debe ser única. primary kaay

conexion.commit() #guardamos lo que hicimos
conexion.close() #cerramos la conexion
print("Tablas creadas correctamente.")

Tablas creadas correctamente.


In [17]:
if os.path.exists("libreria.db"):
    os.remove("libreria.db")
    print("Base de datos vieja eliminada.")

Base de datos vieja eliminada.


In [ ]:

# --- INSERCIÓN DE DATOS ---
conexion = sqlite3.connect("libreria.db")
cursor = conexion.cursor()

# Insertar categorías (una sola vez)
for cat in filas_de_categorias:
    cursor.execute("INSERT OR IGNORE INTO categorias (nombre) VALUES (?);", (cat["nombre"],)) #Es un marcador que indica que un valor externo será sustituido en ese lugar.
    #Si la inserción provoca un error (ej: el autor ya existe), el comando simplemente falla en silencio en lugar de detener el programa.

# Insertar autores y libros
for libro in fila_de_libros:
    # Insertar categoría (buscar ID)
    cursor.execute("SELECT id FROM categorias WHERE nombre = ?;", (libro["categoria"],))
    categoria_id = cursor.fetchone()[0] #me trae una tupla con id y nombre y solo necesito el id o sea el numero 
    
    # Insertar libro
    cursor.execute("""
    INSERT INTO libros (titulo, precio, disponibilidad, calificacion, categoria_id)
    VALUES (?, ?, ?, ?, ?);
    """, (libro["titulo"], libro["precio"], libro["disponibilidad"], libro["calificacion"], categoria_id))
    # Obtener ID del libro recién insertado
    libro_id = cursor.lastrowid
    
    #iterar sobre la lista de autores
    for nombre_autor in libro["autores_lista"]:
        #Buscar el ID del autor existente
        cursor.execute("SELECT id FROM autores WHERE nombre = ?", (nombre_autor,))
        resultado_autor = cursor.fetchone()
        
        if resultado_autor:
            # El autor ya existe, reutilizamos su ID
            autor_id = resultado_autor[0]
        
        else:
            # El autor es nuevo, lo insertamos (INSERT OR IGNORE)
            cursor.execute("INSERT OR IGNORE INTO autores (nombre) VALUES (?)", (nombre_autor,))
            # Obtenemos el ID del autor recién insertado
            autor_id = cursor.lastrowid
        
        # Esto inserta una fila por CADA autor para el mismo libro_id
        cursor.execute("""
        INSERT OR IGNORE INTO libro_autor (libro_id, autor_id)
        VALUES (?, ?);
        """, (libro_id, autor_id))
            
conexion.commit()

print("Datos insertados correctamente.")
conexion.close()

Datos insertados correctamente.


In [ ]:

# Conexión
conexion = sqlite3.connect("libreria.db")

print("--- 5 PRIMEROS LIBROS CON SU AUTOR Y CATEGORÍA ---")
# Consulta con JOINs para verificar la relación de Libros, Autores y Categorías
verificacion = """
    SELECT 
        titulo, 
        precio, 
        calificacion, 
        nombre
    FROM 
        libros,
        categorias
    WHERE
        libros.categoria_id = categorias.id
    LIMIT 5;
    """
df_verificacion = pd.read_sql_query(verificacion, conexion) #Es el método que lee una consulta SQL.

# Muestra el DataFrame, que ahora debería funcionar gracias a 'tabulate'
print(df_verificacion.to_markdown(index=False))  #Método del DataFrame que convierte la tabla de datos a formato Markdown.
# el index es un parámetro opcional que le dice a la función que no incluya el índice numérico de las filas de Pandas en la salida.

conexion.close()

--- 5 PRIMEROS LIBROS CON SU AUTOR Y CATEGORÍA ---
| titulo                                                              |   precio |   calificacion | nombre   |
|:--------------------------------------------------------------------|---------:|---------------:|:---------|
| It's Only the Himalayas                                             |    45.17 |              2 | Travel   |
| Full Moon over Noahâs Ark: An Odyssey to Mount Ararat and Beyond                                                                     |    49.43 |              4 | Travel   |
| See America: A Celebration of Our National Parks & Treasured Sites  |    48.87 |              3 | Travel   |
| Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel |    36.94 |              2 | Travel   |
| Under the Tuscan Sun                                                |    37.33 |              3 | Travel   |


In [4]:

# 1. Conexión a la base de datos
conexion = sqlite3.connect("libreria.db")

# Lista para almacenar los resultados y comentarios
reporte = []

#CONSULTA 1: Libros con 5 Estrellas
consulta1 = """
    SELECT titulo, precio
    FROM  libros
    WHERE calificacion = 5
    LIMIT 5;
    """
reporte.append(("1. 5 Libros con 5 Estrellas ", consulta1))

# CONSULTA 2: Libros de Misterio Baratos 
# Propósito: Encontrar libros de un género específico que sean económicos (menos de £15).
consulta2 = """
    SELECT libros.titulo, libros.precio
    FROM libros, categorias
    WHERE libros.categoria_id = categorias.id AND categorias.nombre = 'Mystery' AND libros.precio < 15.00
    LIMIT 5;
    """
reporte.append(("2. Libros de 'Mystery' más Baratos que £15", consulta2))


#  CONSULTA 3: Total de Libros en la Categoría 'Fiction' 
# Propósito: Medir la popularidad o volumen de un género en la tienda.
consulta3 = """
    SELECT COUNT(libros.id) AS total_libros_ficcion
    FROM libros, categorias
    WHERE libros.categoria_id = categorias.id AND categorias.nombre = 'Fiction';
    """
reporte.append(("3. Cantidad Total de Libros en la Categoría 'Fiction'", consulta3))


# CONSULTA 4: Libros más Baratos 
consulta4 = """
    SELECT titulo, precio
    FROM  libros
    ORDER BY precio ASC  -- ASC (Ascendente) ordena del más bajo al más alto
    LIMIT 5;
    """
reporte.append(("4. Los 5 Libros con el Precio Más Barato:", consulta4))

# --- CONSULTA 5: Top 5 Libros Más Caros ---
consulta5 = """
    SELECT titulo, precio
    FROM libros
    ORDER BY precio DESC 
    LIMIT 5;
    """
reporte.append(("5. Los 5 Libros con el Precio Más Alto", consulta5))

#  Ejecución e Impresión del Reporte 
print("\t\tREPORTE DE 5 CONSULTAS ÚTILES (Forma Sencilla)")

for titulo, consulta in reporte:
    print(f"\n{titulo}:")
    print("-" * (len(titulo) + 5))

    # Ejecuta la consulta
    df_resultado = pd.read_sql_query(consulta, conexion)
    
    # Imprime la tabla formateada
    print(df_resultado.to_markdown(index=False))

conexion.close()

		REPORTE DE 5 CONSULTAS ÚTILES (Forma Sencilla)

1. 5 Libros con 5 Estrellas :
---------------------------------
| titulo                                                                   |   precio |
|:-------------------------------------------------------------------------|---------:|
| 1,000 Places to See Before You Die                                       |    26.08 |
| A Time of Torment (Charlie Parker #14)                                   |    48.35 |
| What Happened on Beale Street (Secrets of the South Mysteries #2)        |    25.37 |
| The Bachelor Girl's Guide to Murder (Herringford and Watts Mysteries #1) |    52.3  |
| The Silkworm (Cormoran Strike #2)                                        |    23.05 |

2. Libros de 'Mystery' más Baratos que £15:
-----------------------------------------------
| titulo                                 |   precio |
|:---------------------------------------|---------:|
| That Darkness (Gardiner and Renner #1) |    13.92 |
| Tastes Like F

In [47]:

# Conexión a la base de datos
conexion = sqlite3.connect("libreria.db")
cursor = conexion.cursor()

# Consulta de prueba: buscar libros cuyo título empiece con 'The'
consulta_busqueda = "SELECT titulo, precio FROM libros WHERE titulo LIKE 'The%'; "

print("--- 1. ANTES DEL ÍNDICE ---")

# MEDIR TIEMPO SIN ÍNDICE
tiempo_inicio = time.time()
cursor.execute(consulta_busqueda).fetchall() #Se usa aquí para asegurar que todo el trabajo de la consulta (la búsqueda en la BD y la transferencia de datos a Python) se complete antes de que se tome el tiempo final
tiempo_sin_indice = time.time() - tiempo_inicio

print(f"Tiempo de búsqueda sin índice: {tiempo_sin_indice:.6f} segundos")

print("\n--- 2. CREANDO EL ÍNDICE ---")

# CREAR EL ÍNDICE: Le dice a la BD que ordene internamente la columna 'titulo'
try:
    cursor.execute("CREATE INDEX idx_titulo ON libros (titulo);") #inicia la acción de construir la estructura de datos ordenada
    conexion.commit()
    print("   Índice 'idx_titulo' creado con éxito en la columna 'titulo'.")
except sqlite3.OperationalError:
    print("   (Advertencia: El índice ya existía. Continuando...)")

print("\n--- 3. DESPUÉS DEL ÍNDICE (Consulta Rápida) ---")

# MEDIR TIEMPO CON ÍNDICE
tiempo_inicio = time.time()
cursor.execute(consulta_busqueda).fetchall()
tiempo_con_indice = time.time() - tiempo_inicio

print(f"Tiempo de búsqueda CON índice: {tiempo_con_indice:.6f} segundos")

print("\n--- 4. CONCLUSIÓN ---")
print(f"   Mejora de velocidad: {tiempo_sin_indice:.6f}s  vs  {tiempo_con_indice:.6f}s")

if tiempo_con_indice < tiempo_sin_indice:
    mejora = (tiempo_sin_indice / tiempo_con_indice)
    print(f"   ¡La consulta es ahora {mejora:.1f} veces más rápida!")
else:
    print("   La diferencia no fue significativa, pero la lógica de optimización es correcta.")

conexion.close()

--- 1. ANTES DEL ÍNDICE ---
Tiempo de búsqueda sin índice: 0.001828 segundos

--- 2. CREANDO EL ÍNDICE ---
   (Advertencia: El índice ya existía. Continuando...)

--- 3. DESPUÉS DEL ÍNDICE (Consulta Rápida) ---
Tiempo de búsqueda CON índice: 0.001097 segundos

--- 4. CONCLUSIÓN ---
   Mejora de velocidad: 0.001828s  vs  0.001097s
   ¡La consulta es ahora 1.7 veces más rápida!


In [21]:
conexion = sqlite3.connect("libreria.db")

cursor = (
    """
    SELECT categorias.nombre, COUNT(DISTINCT autores.id) AS Total_autores
    
    FROM categorias 
    
    JOIN libros ON categorias.id = libros.categoria_id
    
    JOIN libro_autor ON libros.id = libro_autor.libro_id
    
    JOIN autores ON libro_autor.autor_id = autores.id
    
    GROUP BY (categorias.nombre)

    ORDER BY Total_autores DESC
    """
)

df_verificacion = pd.read_sql_query(cursor, conexion)

print(df_verificacion.to_markdown(index=False)) 

| nombre             |   Total_autores |
|:-------------------|----------------:|
| Default            |              86 |
| Nonfiction         |              49 |
| Fiction            |              49 |
| Add a comment      |              43 |
| Young Adult        |              33 |
| Classics           |              27 |
| Childrens          |              26 |
| Historical Fiction |              24 |
| Sequential Art     |              17 |
| Poetry             |              16 |
| History            |              14 |
| Science Fiction    |              11 |
| Romance            |              11 |
| Mystery            |              11 |
| Horror             |              11 |
| Music              |              10 |
| Fantasy            |              10 |
| Autobiography      |              10 |
| Philosophy         |               9 |
| Womens Fiction     |               8 |
| Science            |               8 |
| Business           |               8 |
| Thriller      

In [25]:
conexion = sqlite3.connect("libreria.db")

cursor = """
    SELECT categorias.nombre, MIN(libros.precio) AS precio_minimo
    FROM categorias
    JOIN libros ON categorias.id = libros.categoria_id
    GROUP BY categorias.nombre
    ORDER BY precio_minimo ASC
"""

df_verificacion = pd.read_sql_query(cursor, conexion)

print(df_verificacion.to_markdown(index=False)) 

| nombre             |   precio_minimo |
|:-------------------|----------------:|
| Young Adult        |           10    |
| Science            |           10.01 |
| Add a comment      |           10.02 |
| Sequential Art     |           10.16 |
| Default            |           10.23 |
| Art                |           10.29 |
| Psychology         |           10.4  |
| Horror             |           10.56 |
| Fiction            |           10.6  |
| Childrens          |           10.62 |
| Science Fiction    |           10.65 |
| Mystery            |           10.69 |
| Autobiography      |           10.93 |
| Crime              |           10.97 |
| Food and Drink     |           11.05 |
| Romance            |           11.1  |
| History            |           11.45 |
| Nonfiction         |           11.68 |
| Humor              |           11.83 |
| Self Help          |           12.08 |
| Fantasy            |           12.16 |
| Music              |           12.36 |
| Business      

In [27]:

conexion = sqlite3.connect("libreria.db")

cursor = """
    SELECT categorias.nombre, AVG(libros.calificacion) AS prom_cali
    FROM categorias
    JOIN libros ON categorias.id = libros.categoria_id
    GROUP BY categorias.nombre
    ORDER BY prom_cali DESC




"""

df_verificacion = pd.read_sql_query(cursor, conexion)

print(df_verificacion.to_markdown(index=False)) 

| nombre             |   prom_cali |
|:-------------------|------------:|
| Novels             |     5       |
| Erotica            |     5       |
| Adult Fiction      |     5       |
| Christian Fiction  |     4.16667 |
| Health             |     3.75    |
| Art                |     3.625   |
| Poetry             |     3.52632 |
| Humor              |     3.4     |
| Spirituality       |     3.33333 |
| Young Adult        |     3.2963  |
| Historical Fiction |     3.23077 |
| Fiction            |     3.18462 |
| New Adult          |     3.16667 |
| Music              |     3.15385 |
| Religion           |     3.14286 |
| Womens Fiction     |     3.11765 |
| Fantasy            |     3.08333 |
| Suspense           |     3       |
| Sports and Games   |     3       |
| Historical         |     3       |
| Autobiography      |     3       |
| Sequential Art     |     2.97333 |
| History            |     2.94444 |
| Mystery            |     2.9375  |
| Science            |     2.92857 |
|